# Simulación del Truco

Las **cartas** deben registra el numero, palo y valor de la carta. 
El valor debe reflejar la jerarquía de las cartas en el juego del truco.

In [78]:
def calcular_valor(numero, palo):
    valores = {
        (1, 'espada'): 14,
        (1, 'basto'): 13,
        (7, 'oro'): 12,
        (7, 'copa'): 11,
        3: 10,
        2: 9,
        1: 8,
        12: 7,
        11: 6,
        10: 5,
        7: 4,
        6: 3,
        5: 2,
        4: 1
    }
    return valores.get((numero, palo), valores.get(numero, 0))

lindo = {'oro': '🌞', 'copa': '🍷', 'espada': '🗡️', 'basto': '🥖'}

class Carta:
    def __init__(self, numero, palo):
        self.numero = numero
        self.palo = palo

    @property
    def valor(self):
        return calcular_valor(self.numero, self.palo)
    def __str__(self):
        return f"{self.numero:2}{lindo[self.palo]}"
    def __repr__(self):
        return self.__str__()

    def __gt__(self, otra):
        return self.valor > otra.valor
    
c = Carta(1, "oro")
print(c.numero) # 1
print(c.palo)  # oro
print(c.valor)  # 8
print(str(c))  # 1 de oro
print(repr(c))
a = Carta(7, "copa")
b = Carta(7, "oro")
print(a, b)  # False

1
oro
8
 1🌞
 1🌞
 7🍷  7🌞


**Mazo**: Representan las cartas españolas de 1 al 12, en los palos de oro, copa, espada y basto.

Debe existir una función que permita barajar las cartas.
y otra función que permita repartir las cartas a los jugadores.

In [79]:
import random

numeros = [1, 2, 3, 4, 5, 6, 7, 10, 11, 12]
palos = ['oro', 'copa', 'espada', 'basto']

class Mazo:
    def __init__(self):
        self.cartas = [Carta(numero, palo) for numero in numeros for palo in palos] 
        self.mezclar()       

    def mezclar(self):
        random.shuffle(self.cartas)
    
    def repartir(self, cantidad):
        return [self.cartas.pop() for _ in range(cantidad)]

m = Mazo()
m.mezclar()
print(m.cartas)
print(m.repartir(3))

[ 4🗡️,  2🍷,  6🍷,  2🌞, 12🍷,  6🌞,  5🌞,  7🍷,  4🍷,  5🍷,  6🗡️,  2🥖,  3🗡️,  5🗡️,  1🌞,  3🍷, 11🥖,  5🥖,  7🌞, 12🗡️,  4🥖, 11🌞, 12🌞,  6🥖, 10🗡️,  1🍷, 10🥖,  7🥖,  1🗡️,  2🗡️, 11🍷, 10🍷, 11🗡️,  4🌞,  1🥖, 10🌞,  3🥖,  3🌞,  7🗡️, 12🥖]
[12🥖,  7🗡️,  3🌞]


**Jugador**: Deben tener un nombre y las cartas que tiene en la mano. Debe acumular los puntos ganados.

Nota: Para simplificar el problema, el jugador elige la carta a jugar de forma aleatoria.

In [80]:
class Jugador:
    
    def __init__(self, nombre):
        self.nombre = nombre
        self.cartas = []
        self.puntos = 0 

    def recibir_cartas(self, cartas):
        self.cartas = cartas
    
    def jugar_carta(self):
        random.shuffle(self.cartas)
        return self.cartas.pop()
    
    def __str__(self):
        return self.nombre
    def __repr__(self) -> str:
        return self.__str__()

jugador = Jugador("Juan")
jugador.recibir_cartas(m.repartir(3))

print(jugador.cartas)
print(jugador.jugar_carta())
print(jugador.cartas)

[ 3🥖, 10🌞,  1🥖]
 3🥖
[ 1🥖, 10🌞]


**Partida**: Debe tener dos jugadores, un mazo de cartas y un puntaje. Debe permitir jugar una carta por ronda y definir el ganador de la ronda. Debe permitir jugar hasta que se algun jugador tenga 15 puntos.

In [85]:
class Partida:
    def __init__(self, jugador1, jugador2):
        self.jugadores = [jugador1, jugador2]
        self.mazo = Mazo()

    def jugar_ronda(self):
        return [[j.jugar_carta() for j in self.jugadores] for _ in range(3)]
    
    def evaluar_jugada(self, jugadas):
        ganadas = [ sum(1 for c1, c2 in jugadas if c1 > c2),
                   sum(1 for c1, c2 in jugadas if c2 > c1)]
        
        for ganadas, jugador in zip(ganadas, self.jugadores):
            if ganadas >= 2:
                jugador.puntos += 1
                print(f"{jugador} gana la ronda")
    def puntajes(self):
        print(f"\nPuntos:")
        for jugador in self.jugadores:
            print(f" - {jugador}: {jugador.puntos}")
        print('-' * 20)

    def jugar(self):
        ronda = 0
        while all(j.puntos < 15 for j in self.jugadores):
            self.mazo = Mazo()
            ronda += 1
            print(f"Ronda {ronda} {[jugador for jugador in self.jugadores]}")

            for jugador in self.jugadores:
                jugador.recibir_cartas(self.mazo.repartir(3))
            
            jugadas = self.jugar_ronda()
            for jugada in jugadas:
                print(' ', jugada)
            self.evaluar_jugada(jugadas)
            
            self.puntajes()

        for jugador in self.jugadores:
            if jugador.puntos >= 15:
                print(f"{jugador} gana la partida")
        
p = Partida(Jugador("👨🏻‍🎓 Carlos"), Jugador("👩🏻‍🔧 Analia"))
p.jugar()

Ronda 1 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [ 1🍷, 11🥖]
  [12🍷,  7🍷]
  [12🥖,  5🗡️]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 1
 - 👩🏻‍🔧 Analia: 0
--------------------
Ronda 2 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [12🗡️,  3🌞]
  [11🌞,  6🍷]
  [ 3🥖,  7🍷]
👩🏻‍🔧 Analia gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 1
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 3 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [ 2🍷,  4🗡️]
  [ 1🍷, 10🗡️]
  [10🍷, 12🌞]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 2
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 4 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [ 7🗡️, 11🍷]
  [ 7🌞,  5🥖]
  [10🍷,  4🥖]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 3
 - 👩🏻‍🔧 Analia: 1
--------------------
Ronda 5 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [ 4🗡️, 12🥖]
  [ 2🥖,  3🥖]
  [ 6🗡️, 10🍷]
👩🏻‍🔧 Analia gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 3
 - 👩🏻‍🔧 Analia: 2
--------------------
Ronda 6 [👨🏻‍🎓 Carlos, 👩🏻‍🔧 Analia]
  [ 5🗡️, 11🥖]
  [ 1🌞, 10🗡️]
  [12🌞,  4🥖]
👨🏻‍🎓 Carlos gana la ronda

Puntos:
 - 👨🏻‍🎓 Carlos: 4
 - 👩🏻‍🔧 Analia: 2
----------------

In [44]:
a, b = [1, 2]
print(a)
print(b)

1
2
